<p align='center'><img src="img/QashChain.png" width="250"></p>

# Qashchain - Quantum Blockchain
##### Note: data stored on this blockchain is not encrypted, anyone with access to valid chain can read/write 
- blockchain powered by quantum hashing circuits
- quantum hash circuits support superconductor and photonic operations
- JAX is used for hardware acceleration
  - only superconductor circuit supports JAX acceleration, photonic circuit does not
- different simulators are compatible based on scenario
  - superconductor:
    - 'default.qubit.jax': default JAX powered simulator
    - 'lightning.gpu': nvidia cuda/cuquantum powered simulator
    - 'lightning.kokkos': kokkos c++ powered simulator (default build is OpenMP, Cuda option is also available)
  - photonic:
    - 'strawberryfields.fock': fock based simulator based on xanadu/strawberryfields
    - 'strawberryfields.gaussian': gaussian simulator based on xanadu/strawberryfields (only gaussian operations supported)
- proper usage would require a hybrid-classical-quantum system
  - classical part handles blockchain functions
  - quantum part handles hashing operations
- this notebook does not cover aspect of deployment
  - blockchain code can be adapted for a centralized or decentralized environment
  - does not implement smart-contracts 
  - does not cover consensus algorithm
  - user access control not implemented
- certain values are only partially used in hashes for preformance purposes
  - timestamp, previous_hash
  - repeated characters such as '-' and '.' are removed from timestamp to increase hash security
  - if number of qubits/wires is 3 than first 3 characters are from particular value are used
  - if computing resources are available, these circuits support full hashes for all block values
  - using entire previous_hash will lead to hash sizes increasing exponentially, therefore it is best to use a fixed number of qubits/wires
    - performance bottleneck will probably arise if entire previous_hash is used vs using fixed qubit/wire number
    - fixed number of qubits/wires will keep hash lengths more consistant

In [1]:
import qkdc_helper as helper
import blockchain as bchain

### Set x64 mode (double precision)
- hashes have more variation using double precision mode
- some JAX features, like JIT, require double precision mode

In [2]:
x64_mode = True   # double precision float mode for JAX 
helper.x64Switch(x64_mode)

jax float64 = enabled


### Check JAX Platform (CPU vs GPU)
##### Note: photonic circuit does not support JAX acceleration
- check whether JAX is using CPU or GPU for acceleration
- GPU usage requires Nvidia graphics card and cuda version of JAX
- if constant-folding error occurs, comment out @partial in super_cirq.py

In [3]:
helper.getBackend()

platform = gpu


### Define Parameters
- string1, string2, string3 represent the data being stored on the blockchain
- circuits: dictionary of possible circuits that can be used
  - circuit type is set by circuit2use
- devices: dictionary of possible simulators that can be used
  - choice of simulator is set by device2use
- pad_length: length of extra padding to apply to data input
  - pad_length=2 is equivalent to adding 2 characters of padding
  - pad_length=0 means no padding is applied
- pepper: like a salt, but for quantum states
  - is an array of floats representing quantum states per qubit/wire
  - ex. jnp.array([0.0, 0.0, 0.0]) is a pepper that applied to 3 qubits/wires
- seed: seed to use for random JAX stuff and Strong Entanglement layer
  - must be integer type
- num_wires: number of qubits/wires needed for quantum operations
  - each qubit/wire corresponds to a character, so a string of length 3 requires 3 qubits/wires => num_wires=3

In [4]:
string1 = 'block1'
string2 = 'block2'
string3 = 'block3'
circuits = {
    's': 'superconductor',
    'p': 'photonic'
}
devices = {
    'default': 'default.qubit.jax',
    'nvidia': 'lightning.gpu',
    'kokkos': 'lightning.kokkos',
    'fock': 'strawberryfields.fock',
    'gaussian': 'strawberryfields.gaussian'
}
circuit2use = circuits['s']
device2use = devices['nvidia']
pad_length = 0
pepper = None # jnp.array([])
seed = 0
num_wires = len(string1) + pad_length

### Convert String Data and Apply Padding
- convert data into array of floats
  - float values are used for quantum operations
- extra padding appends extra values to data to increase data size
  - hashes vary in length based on string length so padding is a good way to keep hash lengths consistant

In [5]:
data1 = helper.createAndPad(string1, pad_length)
data2 = helper.createAndPad(string2, pad_length)
data3 = helper.createAndPad(string3, pad_length)

print(f"data1 => {data1}")
print(f"data2 => {data2}")
print(f"data3 => {data3}")

data1 => [ 98. 108. 111.  99. 107.  49.]
data2 => [ 98. 108. 111.  99. 107.  50.]
data3 => [ 98. 108. 111.  99. 107.  51.]


### Set Enviornment Variables

In [6]:
helper.setEnvVars(device2use)

kokkos is not being used, no enviornment variables need to be set...


### Create 1st Blockchain Ledger
- no previous blockchain file is used/loaded
    - genesis block has to be created since chain is being made from scratch
- adding 2 data blocks to blockchain

In [7]:
ledger1 = bchain.Blockchain(None, device2use, circuit2use)
ledger1.create_genesis_block(num_wires, seed, pepper)
ledger1.add_block(data1, num_wires, seed, pepper)
ledger1.add_block(data2, num_wires, seed, pepper)

device 'lightning.gpu' is now set...
circuit 'superconductor' is now set...
no chain file specified, blank chain is being created...
creating/adding genesis block...
adding block 1...
adding block 2...


### Print 1st Blockchain

In [8]:
ledger1.print_blocks()

...
index: 0
timestamp: 2024-02-20 16:15:51.508270
data: [48. 93. 70. 97. 46. 66.]
previous hash: 28190738112e404bfd2a86e899c2400e6ce1ec40ccc74023f42a8dbfb818402c67f1edd699124043d248c2abb5da
current hash: b5785bc278e0c0d15417922a40399a85185e7fcf3fa07ffdb5068e0040272d7a9437bdee40295576b06c67064051a38d9b8f32d8a348f23701ff4044736d95b2e6474023a4139443be04402a4aad0780a3b7401d4daa5d75df29404bad49d9db320a721ffe6e818840192c5d631e68f64041e848f21e00f84036d244aebfe15a402d16da175b3fc14043d59eeb6dc8bf
...
index: 1
timestamp: 2024-02-20 16:15:53.035405
data: [ 98. 108. 111.  99. 107.  49.]
previous hash: b5785bc278e0c0d15417922a40399a85185e7fcf3fa07ffdb5068e0040272d7a9437bdee40295576b06c67064051a38d9b8f32d8a348f23701ff4044736d95b2e6474023a4139443be04402a4aad0780a3b7401d4daa5d75df29404bad49d9db320a721ffe6e818840192c5d631e68f64041e848f21e00f84036d244aebfe15a402d16da175b3fc14043d59eeb6dc8bf
current hash: c5255496453cc0d15417922a40399a85185e7fcf3fa07ffdb5068e0040272d7a9437bdee40295576b06c67064051a38d9b8

### Print Data Values For Each Block in 1st Blockchain
- this prints the converted data arrays back into strings

In [9]:
print("Ledger 1 Data Values:")
print("---------------------")
for block in ledger1.chain:
    print(f"index:{block.index} => {helper.convert2String(block.data)}")

Ledger 1 Data Values:
---------------------
index:0 => 0]Fa.B
index:1 => block1
index:2 => block2


### Save 1st Blockchain
- save 1st blockchain to blockchain.npy

In [10]:
ledger1.save_chain()

### Create 2nd Blockchain and Add New Block
- previous blockchain is loaded from .npy file
- additional block is added to blockchain

In [11]:
ledger2 = bchain.Blockchain('blockchain.npy', device2use, circuit2use)
ledger2.add_block(data3, num_wires, seed, pepper)

device 'lightning.gpu' is now set...
circuit 'superconductor' is now set...
chain file is loading...
adding block 3...


### Print 2nd Blockchain

In [12]:
ledger2.print_blocks()

...
index: 0
timestamp: 2024-02-20 16:15:51.508270
data: [48. 93. 70. 97. 46. 66.]
previous hash: 28190738112e404bfd2a86e899c2400e6ce1ec40ccc74023f42a8dbfb818402c67f1edd699124043d248c2abb5da
current hash: b5785bc278e0c0d15417922a40399a85185e7fcf3fa07ffdb5068e0040272d7a9437bdee40295576b06c67064051a38d9b8f32d8a348f23701ff4044736d95b2e6474023a4139443be04402a4aad0780a3b7401d4daa5d75df29404bad49d9db320a721ffe6e818840192c5d631e68f64041e848f21e00f84036d244aebfe15a402d16da175b3fc14043d59eeb6dc8bf
...
index: 1
timestamp: 2024-02-20 16:15:53.035405
data: [ 98. 108. 111.  99. 107.  49.]
previous hash: b5785bc278e0c0d15417922a40399a85185e7fcf3fa07ffdb5068e0040272d7a9437bdee40295576b06c67064051a38d9b8f32d8a348f23701ff4044736d95b2e6474023a4139443be04402a4aad0780a3b7401d4daa5d75df29404bad49d9db320a721ffe6e818840192c5d631e68f64041e848f21e00f84036d244aebfe15a402d16da175b3fc14043d59eeb6dc8bf
current hash: c5255496453cc0d15417922a40399a85185e7fcf3fa07ffdb5068e0040272d7a9437bdee40295576b06c67064051a38d9b8

### Print Data Values For Each Block in 2nd Blockchain

In [13]:
print("Ledger 2 Data Values:")
print("---------------------")
for block in ledger2.chain:
    print(f"index:{block.index} => {helper.convert2String(block.data)}")

Ledger 2 Data Values:
---------------------
index:0 => 0]Fa.B
index:1 => block1
index:2 => block2
index:3 => block3
